In [1]:
import os
import sys
import pandas as pd 
import time
import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *


In [7]:
file_path = '/Users/yantingting/Seafile/风控/模型/15 印度/V4模型/Data'

In [3]:
query0 = '''
with table_flag1 as 
(select t1.order_no,t1.product_name,date(t1.start_date) as start_date,
case when t1.overdue_days >= 7 then 1 else 0 end as flag1
from dw_ind_supercore_prod_india_supercore_prod_repayment_schedules t1
left join ind_oss_creditreport t2 
on t1.customer_id = t2.customerid::int	
where customer_type in ('NewApplicationOrder','OrderTypeIndNewClient')
and extend_period = 0 
and date(start_date)>= '2020-01-02'
and date(start_date)<= '2020-03-19'
and t2.customerid is not null),
table_flag2 as (
with loan as (select order_no,start_date,due_date,actual_paid_off_date,extend_period,overdue_days
from dw_ind_supercore_prod_india_supercore_prod_repayment_schedules),
b as (select order_no,min(start_date) as first_start
from dw_ind_supercore_prod_india_supercore_prod_repayment_schedules
group by 1)
select loan.*,date(b.first_start) +55 as day1,
row_number()over(partition by loan.order_no order by loan.start_date desc) as rank1,
case when due_date>=date(b.first_start) +55 then 0
     when due_date<date(b.first_start) +55 and date(actual_paid_off_date) !='1970-01-01' and  actual_paid_off_date<=date(b.first_start) +55 then overdue_days
     when due_date<date(b.first_start) +55 and date(actual_paid_off_date) ='1970-01-01' then date_part('day',date(b.first_start) +55 - due_date)
     when due_date<date(b.first_start) +55 and actual_paid_off_date>date(b.first_start) +55 then date_part('day',date(b.first_start) +55 - due_date)
     end as lastoverdue_days
from loan 
left join b 
on loan.order_no = b.order_no
where start_date<=date(b.first_start) +55)
select 
table_flag1.*,
case when table_flag2.lastoverdue_days >= 7 then 1 else 0 end as flag2,
case when table_flag2.extend_period >0 then 1 else 0 end as flag_extend
from table_flag1
left join table_flag2
on table_flag1.order_no = table_flag2.order_no
where table_flag2.rank1 = 1
'''

In [4]:
time1 = time.time()
df0 =  data_con.database_gn(query0)
time2 = time.time()
print('run_time: ', time2 - time1)
df0.shape
df0.head()

Successful connection
run_time:  1695.560294866562


(33918, 6)

,order_no,product_name,start_date,flag1,flag2,flag_extend
0,Gt20200220004550,GotoCash,2020-02-20,0,0,0
1,Gt20200220001167,GotoCash,2020-02-20,0,0,0
2,Gt20200220000189,GotoCash,2020-02-20,1,1,0
3,Gt20200120010102,GotoCash,2020-01-21,0,0,0
4,Gt20200221009701,GotoCash,2020-02-22,0,0,0


In [8]:
save_data_to_pickle(df0,file_path,'flag.pkl')
print(1)

1
